# Install

In [1]:
! pip install speechbrain==1.0.0 -q
! pip install faster_whisper -q
! pip install pyannote.audio -q
! pip install whisper -q
! pip install datasets -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 760.1/760.1 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.8/117.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 722.2/722.2 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.1/33.1 MB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 81.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   

In [2]:
import librosa
import traceback
from faster_whisper import WhisperModel
import torch
import datasets
from pathlib import Path
import pandas as pd
import re
import time
import os
import numpy as np
from sklearn.cluster import AgglomerativeClustering
from pyannote.audio.pipelines.speaker_verification import PretrainedSpeakerEmbedding
from pyannote.audio import Audio
from pyannote.core import Segment
import speechbrain
from scipy.spatial.distance import cdist

In [3]:
import datetime

# UPLOAD AUDIO

In [4]:
# Get the path to the audio fine
audio_file_path="/content/drive/MyDrive/@Degree/Sep-2024/DLP/out.mp3"

## .mp3 to .wav conversion

* sample rate = 16 KHz
* Channel -1 (mono)
* Audio Codec = pcm_s16le

In [5]:
# mp3 to wav conversion
! ffmpeg -i "{audio_file_path}" -ar 16000 -ac 1 -c:a pcm_s16le "{audio_file_path[:-4]}.wav"


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

# SPEAKER DIARIZATION - USING WHISPER SEGMENTS AND AGGLOMERATIVE HIERARCHICAL CLUSTERING

In [6]:
# whisper model different sizes
whisper_models = ["tiny", "base", "small", "medium", "large-v1", "large-v2"]

# get the voxceleb embedding
embedding_model=PretrainedSpeakerEmbedding(
    "speechbrain/spkrec-ecapa-voxceleb",
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
)

hyperparams.yaml:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/speechbrain/utils/autocast.py:63: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  wrapped_fwd = torch.cuda.amp.custom_fwd(fwd, cast_inputs=cast_inputs)


embedding_model.ckpt:   0%|          | 0.00/83.3M [00:00<?, ?B/s]

mean_var_norm_emb.ckpt:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

classifier.ckpt:   0%|          | 0.00/5.53M [00:00<?, ?B/s]

label_encoder.txt:   0%|          | 0.00/129k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/speechbrain/utils/checkpoints.py:152: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(path, map_location=device), strict=Fal

In [7]:
def convert_time(secs):
    return datetime.timedelta(seconds=round(secs))

def segment_embedding(segment, duration):
    try:
        audio=Audio()

        start=segment["start"]
        end=min(duration, segment["end"])

        clip=Segment(start, end)
        waveform, sample_rate=audio.crop(audio_file, clip)

        embeddings=embedding_model(waveform[None])
        print(embeddings)
        return embeddings
    except Exception as e:
        traceback.print_exc()
        raise RuntimeError("Error During Segment Embedding", e)


In [8]:
model=WhisperModel("medium", compute_type="int8")
time_start=time.time()

config.json:   0%|          | 0.00/2.26k [00:00<?, ?B/s]

vocabulary.txt:   0%|          | 0.00/460k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.20M [00:00<?, ?B/s]

model.bin:   0%|          | 0.00/1.53G [00:00<?, ?B/s]

In [9]:
audio_data, sampling_rate=librosa.load(audio_file, mono=True, sr=16000)
duration=len(audio_data)/sampling_rate # calculate duration
# transcribe audio
options = dict(language="hi", beam_size=5, best_of=5)
transcribe_options=dict(task="transcribe", **options)
segments_raw, info=model.transcribe(audio_file, **transcribe_options)

NameError: name 'audio_file' is not defined

In [ ]:
segments = []
for segment_chunk in segments_raw:
    chunk={}
    chunk["start"]=segment_chunk.start
    chunk["end"]=segment_chunk.end
    segments.append(chunk)
    print(segment_chunk.start, segment_chunk.end, segment_chunk.text)

0.0 2.94  एक पूरा एको सिस्टम जो है वो इसमें लग जाता है
2.94 5.7  उदर से ये प्रश्ने नहीं पूछा जाता कई बात
5.7 7.5  या पूछने पर भी जबाब नहीं मिलता
7.5 11.0  कि बीजेपी के गोशना पतर में कहां है कि समिदान खतम करेंगे
11.0 12.5  अगला चुनाव नहीं होगा
13.5 17.1  आगला चुनाव ही नहीं आइगे तो नहीं कराएंगे
17.1 18.5  आरक्षन कर देंगे
18.5 22.1  लिखी चीजों पर भी जूट पहला जाता आप पर बी जिकर किया
22.1 26.5  तीन क्रिशी कानून जिस्में का अग्या कि किसानों की जमीन चीन ले जाएगी
26.5 28.5  वो कही नहीं लिखा था
28.5 32.5  लेकिन वो भ्रहम फ़ला फ़ला कर फ़ला फ़ला कर चीजे जो हैं वो खराब की गई
32.5 39.5  आप जब इस बार ये कह रहे हैं बार बार कि मोदि मर जाएगा लेकिन धर्म के नाम पर आरक्षण नहीं आने देगा
39.5 43.5  आपको क्या खतरा दिकता है अगर ध्रम के नाम पर आरक्षण आए या मुसल्मानों को आरक्षण देगा


In [ ]:
def speech_to_text(audio_file, whisper_model):
    model=WhisperModel(whisper_model, compute_type="int8")
    time_start=time.time()

    try:
        # get duration
        audio_data, sampling_rate=librosa.load(audio_file, mono=True, sr=16000)
        duration=len(audio_data)/sampling_rate # calculate duration
        # transcribe audio
        options = dict(language="hi", beam_size=5, best_of=5)
        transcribe_options=dict(task="transcribe", **options)
        segments_raw, info=model.transcribe(audio_file, **transcribe_options)

        # convert back to original format
        segments = []
        for segment_chunk in segments_raw:
            chunk={}
            chunk["start"]=segment_chunk.start
            chunk["end"]=segment_chunk.end
            segments.append(chunk)
            print(segment_chunk.start, segment_chunk.end, segment_chunk.text)

    except Exception as e:
        raise RuntimeError("Error converting video to audio")

    try:
        # create embedding
        embeddings = np.zeros(shape=(len(segments), 192))
        for i, segment in enumerate(segments):
            embeddings[i]=segment_embedding(segment, duration)
        embedddings=np.nan_to_num(embeddings)

        # assign speaker label

        best_num_speaker=3
        clustering = AgglomerativeClustering(best_num_speaker).fit(embeddings)
        labels=clustering.labels_
        for i in range(len(segments)):
            segments[i]["speaker"] = "SPEAKER"+str(labels[i]+1)

        # Make output
        objects = {
            "Start":[],
            "End": [],
            "Speaker": []
        }

        for (i, segment) in enumerate(segments):
            if i == 0 or segments[i-1]["speaker"] in segment["speaker"]:
                objects["Start"].append((convert_time(segment["start"])))
                objects["speaker"].append(segment["speaker"])

                if i != 0:
                    objects["End"].append(str(convert_time(segments[i-1]["End"])))
        objects["End"].append(str(convert_time(segments[i-1]["End"])))

        save_path='/contents/test1.csv'
        df_results = pd.DataFrame(objects)
        df_results.to_csv(save_path)
        return df_results, save_path
    except Exception as e:
        raise RuntimeError("Error Running Inference with Local model", e)

audio_file="/content/drive/MyDrive/@Degree/Sep-2024/DLP/out.wav"
selected_whisper_model = "medium"

# Run the transcription
transcription_results, save_path = speech_to_text(audio_file, selected_whisper_model)

print(transcription_results)
print(f"Transcription results saved at: {save_path}")


hyperparams.yaml:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/speechbrain/utils/autocast.py:63: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  wrapped_fwd = torch.cuda.amp.custom_fwd(fwd, cast_inputs=cast_inputs)


embedding_model.ckpt:   0%|          | 0.00/83.3M [00:00<?, ?B/s]

mean_var_norm_emb.ckpt:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

classifier.ckpt:   0%|          | 0.00/5.53M [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/2.20M [00:00<?, ?B/s]

vocabulary.txt:   0%|          | 0.00/460k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.26k [00:00<?, ?B/s]

0.0 2.94  एक पूरा एको सिस्टम जो है वो इसमें लग जाता है
2.94 5.7  उदर से ये प्रश्ने नहीं पूछा जाता कई बात
5.7 7.5  या पूछने पर भी जबाब नहीं मिलता
7.5 11.0  कि बीजेपी के गोशना पतर में कहां है कि समिदान खतम करेंगे
11.0 12.5  अगला चुनाव नहीं होगा
12.5 15.5  यहां वान निशन वान इलक्षन की बात चल्ग है लेकिन कहा जाता है कि
15.5 17.5  अगला चुनाव ही मोदि आइगे तो नहीं कराइगे
17.5 18.5  आरक्षन कर देगे
18.5 22.5  लिखी चीजों पर भी जूट फ़ला जा जाता आप नहीं दिखा दिखा
53.5 56.5  इसमें मोदि मुद्धा ही नहीं है
56.5 60.5  जिस समय सम्विदान बन रहा था
60.5 64.5  देश के गणमान्य लोग समिदान समाम वेटे थे


RuntimeError: ('Error Running Inference with Local model', TypeError("'secons' is an invalid keyword argument for __new__()"))

In [ ]:
! pip install moviepy pandas pillow

In [ ]:
import pandas as pd
from moviepy.editor import VideoFileClip, ImageClip, CompositeVideoClip
from PIL import Image, ImageDraw, ImageFont

df_results = transcription_results

# Step 4: load the video

video_path="/content/videoplayback.mp4" # update with oyur video path
video = VideoFileClip(video_path)

# functions to create an image with text

def create_text_image(text, font_size=70, img_size=(640, 80), bg_color=(0,0,0), text_color=(255,255,255)):
    img=Image.new("RGB", img_size, color=bg_color)
    d = ImageDraw.Draw(img)

    try:
        font=ImageFont.truetype("arial.ttf", font_size)
    except IOError:
        font = ImageFont.load_default()
    text_width, text_height=d.text_size(text, font=font)
    position = ((img[0]-text_width)/2, (img_size[1]-text_height)/2)
    d.text(position, text, fill=text_color, font=font)
    return img

# Step 5: Overlay Speaker Labels

clips = [video]

for _, row in df_results.iterrows():
    start_time = pd.to_datetime(row["Start"]).time()
    end_time = pd.to_datetime(row["End"]).time()

    start_seconds=start_time.hour * 3600 + start_time.minute * 60 + start_time.second
    end_seconds=end_time.hour * 3600 + end_time.minute * 60 + end_time.second

    text_img = create_text_image(row["Speaker"])
    text_img_path="/content/temp_text_img.png"
    text_img.save(text_img_path)

    txt_clip = (ImageClip(text_img_path)
                .set_position('center', 'bottom')
                .set_start(start_seconds)
                .set_duration(end_seconds-start_seconds))

    clips.append(txt_clip)

# Combine All Clips

final_video=CompositeVideoClip(clips)

final_video_path = "/content/videoplayback_label.mp4"
final_video.write_videofile(final_video_path, codec="libx264")

In [ ]:
from IPython.display import HTML
from base64 import b64encode

def show_video(final_video_path, video_width=1000):

    video_file = open(final_video_path, "r+b").read()

    video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"

    return HTML(f"""<video width={video_width} controls><source src="{video_url}"></video>""")

show_video(final_video_path)